# 代理组件: 本地控制代理 (Local Control Agent)

`本地控制代理 (LocalControlAgent)` 是一个**控制代理**，它充当控制算法（如`PID控制器`）和`消息总线`之间的桥梁。它是一个纯粹的事件驱动代理；其逻辑由传入的消息触发，而不是由`run()`循环触发。

其主要功能是：
1. **订阅**一个`observation_topic`以接收状态更新（例如，来自`数字孪生代理`）。
2. **传递**消息中的相关信息给其内部的控制器。
3. **发布**控制器计算出的动作到一个`action_topic`，物理组件（如`闸门`或`阀门`）可以监听该主题。

## 仿真示例

在这个仿真中，我们将创建一个封装了`PIDController`的`LocalControlAgent`。然后，我们将扮演“系统的其余部分”：
1. 手动向代理的观测主题发布状态消息，模拟一个`数字孪生代理`。
2. 在代理的动作主题上监听，以捕获它响应发布的控制信号。

In [ ]:
import pandas as pd
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.control.pid_controller import PIDController
from swp.central_coordination.collaboration.message_bus import MessageBus

# 1. 设置组件
message_bus = MessageBus()
OBSERVATION_TOPIC = "sensor.water_level"
ACTION_TOPIC = "actuator.gate.opening"

# 该代理将使用的控制器
pid_controller = PIDController(Kp=-0.5, Ki=-0.1, Kd=0.0, setpoint=10.0, min_output=0.0, max_output=1.0)

# 代理本身
lca = LocalControlAgent(
    agent_id="gate_controller_agent",
    controller=pid_controller,
    message_bus=message_bus,
    observation_topic=OBSERVATION_TOPIC,
    observation_key='level', # 它将在消息中寻找'level'键
    action_topic=ACTION_TOPIC,
    dt=1.0
)

# 2. 创建一个监听器以接收代理的输出
captured_actions = []
def action_listener(message):
    captured_actions.append(message)

message_bus.subscribe(ACTION_TOPIC, action_listener)

# 3. 仿真循环：手动发布传感器读数
print("--- 手动发布传感器读数 ---")
process_variable_history = [15.0, 14.0, 12.5, 11.0, 10.1, 10.0]

for pv in process_variable_history:
    print(f"发布观测值: {{'level': {pv}}}")
    # 此消息模拟了DigitalTwinAgent的广播
    message_bus.publish(OBSERVATION_TOPIC, {'level': pv, 'timestamp': '...'}) 

print("\n--- 仿真结束 ---")

## 结果

下面是我们的监听器捕获到的动作消息。对于我们发布的每一个观测值，`LocalControlAgent`都被触发，运行其PID控制器，并将产生的`control_signal`发布到动作主题。这演示了完整的事件驱动工作流。

In [ ]:
if not captured_actions:
    print("没有捕获到任何动作。")
else:
    print(f"从主题 '{ACTION_TOPIC}' 捕获到 {len(captured_actions)} 个动作:\n")
    # 创建一个DataFrame以便更好地格式化输出
    df = pd.DataFrame(captured_actions)
    df['observed_level'] = process_variable_history
    print(df[['observed_level', 'control_signal']])